In [17]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [18]:
F_selection = pd.read_csv("F_Selection/top_N_features_SelectFromModel_RFAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [19]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [20]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_14128\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [21]:
top = top_10

rank_data_10 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressor): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.4570160931174088
R-squared (RandomForest Regressor): 0.6052487015218588


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,28.690
13894,Dikembe Mutombo,1,C,39.0,28.300
13740,Hakeem Olajuwon,-1,C,0.0,28.105
13506,Tom Chambers,-1.0,PF,0.0,7.880
13866,Sean Higgins,-1.0,SF,0.0,6.215
13889,Tim Duncan,5T,PF,4.0,4.120
13836,Michael Jordan,4,SG,6.0,2.100
13637,Dennis Rodman,5T,PF,4.0,1.660
13605,David Robinson,3,C,10.0,1.625
13628,Zydrunas Ilgauskas,-1.0,C,0.0,1.460


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 11.592800316455696
R-squared (RandomForest Regressor): 0.31748821314107367


,Player_name,Rank,Pos,Points_won,Predicted_points
13182,Dikembe Mutombo,2,C,10.0,26.845
13424,Gary Payton,3,PG,6.0,25.490
13039,Hakeem Olajuwon,7T,C,1.0,24.395
13269,Alonzo Mourning,1,C,89.0,23.710
13434,David Robinson,4,C,3.0,2.275
13387,Gary Grant,-1.0,PG,0.0,1.735
13056,Trevor Winter,-1.0,C,0.0,1.390
13102,Adonis Jordan,-1.0,PG,0.0,0.895
13461,Peter Aluma,-1.0,C,0.0,0.835
13070,Tyson Wheeler,-1.0,PG,0.0,0.695


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 8.867781303418804
R-squared (RandomForest Regressor): 0.08541613843329254


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,117.495
1046,Sekou Doumbouya,-1.0,PF,0.0,96.065
1184,Nikola Jokić,-1.0,C,0.0,91.410
610,Michael Porter Jr.,-1.0,SF,0.0,69.705
1026,Myles Turner,-1.0,C,0.0,48.705
1123,Draymond Green,10,PF,2.0,23.175
1275,Stephen Curry,-1.0,PG,0.0,18.125
992,Paul George,-1.0,PF,0.0,17.360
813,Ricky Rubio,-1.0,PG,0.0,16.930
997,Jaren Jackson Jr.,5.0,PF,99.0,15.820


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 319.5929200328407
R-squared (RandomForest Regressor): 0.248930596208641


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,56.665
76,Mac McClung,-1.0,SG,0.0,46.475
323,Brook Lopez,2.0,C,309.0,46.075
206,Nic Claxton,9T,C,7.0,44.440
208,Walker Kessler,-1.0,C,0.0,32.895
75,Myles Turner,-1.0,C,0.0,20.760
137,Evan Mobley,3.0,PF,101.0,18.380
197,Kawhi Leonard,-1,SF,0.0,13.240
396,Rudy Gobert,-1.0,C,0.0,8.440
564,Trey Murphy III,-1.0,SF,0.0,4.420


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,2,5T,-1,1,11,1,1,1,-1,...,1.0,2,3,2.0,1.0,1.0,1.0,1.0,3.0,1.0
1,1,3,3T,-1,-1,11,2,-1,-1,6,...,7.0,1,1,1,15T,2.0,3.0,9.0,-1.0,-1.0
2,-1,7T,-1,1,3T,-1,-1,9,4,1,...,11,4,2,-1,6,5.0,-1.0,-1.0,-1.0,2.0
3,-1.0,1,1,1,-1,1,-1,-1,4,3,...,10.0,3.0,5.0,3,-1.0,-1,11,-1.0,-1.0,9T
4,-1.0,4,-1,-1,-1,3,-1,-1,8,-1.0,...,-1.0,8,-1.0,-1.0,3,-1.0,2,5.0,-1.0,-1.0
5,5T,-1.0,2,11T,-1,4,6,-1,-1,4,...,-1,5.0,10,5T,-1.0,-1,10.0,3,10,-1.0
6,4,-1.0,-1.0,-1.0,-1.0,-1,7,10,-1.0,-1,...,3.0,-1.0,4.0,-1,14.0,-1,8T,-1,-1.0,3.0
7,5T,-1.0,-1,3,-1,-1.0,5,18T,-1.0,-1,...,-1.0,15T,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
8,3,-1.0,3T,5T,-1,2,-1,4,5,13T,...,-1.0,7.0,-1.0,-1,-1,6T,-1,-1,-1.0,-1.0
9,-1.0,-1.0,7,5T,-1,-1.0,4,-1,10T,-1.0,...,5,10T,-1.0,-1.0,-1.0,6T,6T,-1,5.0,-1.0


In [22]:
top = top_20

rank_data_20 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressor): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.3706259615384617
R-squared (RandomForest Regressor): 0.6191283894539057


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,29.210
13740,Hakeem Olajuwon,-1,C,0.0,28.500
13894,Dikembe Mutombo,1,C,39.0,28.395
13506,Tom Chambers,-1.0,PF,0.0,6.650
13866,Sean Higgins,-1.0,SF,0.0,5.475
13889,Tim Duncan,5T,PF,4.0,5.075
13836,Michael Jordan,4,SG,6.0,4.005
13605,David Robinson,3,C,10.0,3.180
13666,Brevin Knight,11T,PG,1.0,2.040
13687,Joe Dumars,-1,SG,0.0,1.970


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 11.917808069620254
R-squared (RandomForest Regressor): 0.2983537834691955


,Player_name,Rank,Pos,Points_won,Predicted_points
13182,Dikembe Mutombo,2,C,10.0,27.800
13424,Gary Payton,3,PG,6.0,27.535
13039,Hakeem Olajuwon,7T,C,1.0,27.135
13269,Alonzo Mourning,1,C,89.0,26.565
13488,Tim Duncan,5T,PF,2.0,6.325
13445,Kevin Garnett,7T,PF,1.0,6.040
13121,Bo Outlaw,-1,PF,0.0,5.995
13434,David Robinson,4,C,3.0,5.405
13087,Horace Grant,-1,PF,0.0,5.230
13450,Theo Ratliff,5T,PF,2.0,5.195


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 8.438983493589744
R-squared (RandomForest Regressor): 0.12964045377512612


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,105.390
1046,Sekou Doumbouya,-1.0,PF,0.0,97.845
1184,Nikola Jokić,-1.0,C,0.0,93.105
610,Michael Porter Jr.,-1.0,SF,0.0,75.725
1026,Myles Turner,-1.0,C,0.0,47.410
992,Paul George,-1.0,PF,0.0,38.265
933,Al Horford,9,C,3.0,35.820
1123,Draymond Green,10,PF,2.0,29.975
1212,Jayson Tatum,-1.0,SF,0.0,27.160
1007,Joel Embiid,-1,C,0.0,24.745


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 319.5479682676519
R-squared (RandomForest Regressor): 0.24903623651968498


,Player_name,Rank,Pos,Points_won,Predicted_points
76,Mac McClung,-1.0,SG,0.0,64.075
59,Jaren Jackson Jr.,1.0,C,391.0,57.500
323,Brook Lopez,2.0,C,309.0,46.580
206,Nic Claxton,9T,C,7.0,42.470
208,Walker Kessler,-1.0,C,0.0,20.695
137,Evan Mobley,3.0,PF,101.0,18.570
75,Myles Turner,-1.0,C,0.0,17.480
197,Kawhi Leonard,-1,SF,0.0,12.255
396,Rudy Gobert,-1.0,C,0.0,8.420
497,Joel Embiid,9T,C,7.0,5.235


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,2,5T,-1,1,1,1,1,1,-1,...,1.0,2,3,2.0,1.0,1.0,1.0,1.0,3.0,-1.0
1,-1,3,3T,1,-1,11,2,-1,-1,6,...,7.0,1,1,5T,6,2.0,11,9.0,-1.0,1.0
2,1,7T,-1,1,-1,11,-1,-1,4,13T,...,5,8,5.0,1,15T,5.0,-1.0,-1.0,-1.0,2.0
3,-1.0,1,1,-1,3T,-1,6,9,4,3,...,13T,4,2,-1.0,3,-1.0,3.0,-1.0,-1.0,9T
4,-1.0,5T,2,-1,-1,4,-1,-1,13T,-1.0,...,13T,15T,11T,-1,8.0,-1,2,-1,-1.0,-1.0
5,5T,7T,7,3,-1,3,-1,4,6,-1,...,-1,-1.0,-1.0,-1,-1.0,-1,-1.0,5.0,-1.0,3.0
6,4,-1,-1,2,2,8,7,10,8,1,...,11,5.0,4.0,3,-1,-1.0,10.0,3,9,-1.0
7,3,4,-1,5T,-1,-1,10T,5,-1,4,...,10.0,3.0,10,5T,15T,6T,-1,-1,10,-1
8,11T,-1,-1,-1,3T,13T,5,18T,7,24T,...,-1.0,10T,-1.0,-1,-1.0,6T,8T,-1,-1.0,-1.0
9,-1,5T,8T,11T,-1,-1.0,10T,-1,-1,8,...,-1.0,7.0,-1,-1,-1,-1,-1,-1,-1,9T


In [23]:
top = top_30

rank_data_30 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressor): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.475951012145749
R-squared (RandomForest Regressor): 0.6022065627691036


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,31.605
13740,Hakeem Olajuwon,-1,C,0.0,31.170
13894,Dikembe Mutombo,1,C,39.0,31.100
13506,Tom Chambers,-1.0,PF,0.0,5.645
13889,Tim Duncan,5T,PF,4.0,4.885
13836,Michael Jordan,4,SG,6.0,4.290
13866,Sean Higgins,-1.0,SF,0.0,4.015
13605,David Robinson,3,C,10.0,3.375
13783,Mario Elie,-1,SF,0.0,2.045
13956,Charles Oakley,-1,PF,0.0,1.980


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 11.822181276371309
R-squared (RandomForest Regressor): 0.3039836927016769


,Player_name,Rank,Pos,Points_won,Predicted_points
13182,Dikembe Mutombo,2,C,10.0,30.340
13424,Gary Payton,3,PG,6.0,29.575
13269,Alonzo Mourning,1,C,89.0,28.185
13039,Hakeem Olajuwon,7T,C,1.0,26.980
13434,David Robinson,4,C,3.0,6.425
13488,Tim Duncan,5T,PF,2.0,5.880
13445,Kevin Garnett,7T,PF,1.0,5.460
13121,Bo Outlaw,-1,PF,0.0,4.935
13397,Scottie Pippen,7T,SF,1.0,4.860
13450,Theo Ratliff,5T,PF,2.0,4.650


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 8.185258547008548
R-squared (RandomForest Regressor): 0.15580852597719685


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1046,Sekou Doumbouya,-1.0,PF,0.0,99.595
1022,Rudy Gobert,3.0,C,136.0,90.515
1184,Nikola Jokić,-1.0,C,0.0,83.585
610,Michael Porter Jr.,-1.0,SF,0.0,80.645
1026,Myles Turner,-1.0,C,0.0,46.755
992,Paul George,-1.0,PF,0.0,38.120
1212,Jayson Tatum,-1.0,SF,0.0,29.990
1275,Stephen Curry,-1.0,PG,0.0,26.805
933,Al Horford,9,C,3.0,25.785
813,Ricky Rubio,-1.0,PG,0.0,23.990


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 292.98885119047617
R-squared (RandomForest Regressor): 0.3114523257945334


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,78.350
76,Mac McClung,-1.0,SG,0.0,78.185
323,Brook Lopez,2.0,C,309.0,60.350
206,Nic Claxton,9T,C,7.0,53.975
392,Jacob Gilyard,-1.0,PG,0.0,29.520
137,Evan Mobley,3.0,PF,101.0,28.935
184,Shaquille Harrison,-1.0,SG,0.0,26.990
202,Marcus Smart,-1,PG,0.0,24.435
497,Joel Embiid,9T,C,7.0,20.380
396,Rudy Gobert,-1.0,C,0.0,17.650


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,2,3T,1,1,1,2,-1,1,6,...,1.0,2,3,2.0,15T,2.0,1.0,1.0,-1.0,1.0
1,-1,3,5T,1,3T,-1,-1,-1,13T,-1.0,...,7.0,4,1,1,6,1.0,3.0,9.0,3.0,-1.0
2,1,1,-1,-1,-1,11,1,1,7,13T,...,13T,1,2,5T,1.0,-1,11,-1.0,-1.0,2.0
3,-1.0,7T,1,-1,-1,11,-1,-1,6,3,...,5,15T,5.0,-1,3,5.0,-1.0,-1.0,-1.0,9T
4,5T,4,2,-1,-1,4,-1,-1,-1,-1,...,13T,8,-1.0,3,-1.0,-1.0,-1.0,-1,-1.0,-1.0
5,4,5T,-1,3,-1,3,6,9,-1,1,...,10.0,-1.0,11T,-1.0,-1.0,-1.0,2,5.0,-1.0,3.0
6,-1.0,7T,7,5T,2,8,-1,18T,-1.0,16T,...,-1,3.0,4.0,-1,-1.0,-1,10.0,-1,-1.0,-1.0
7,3,-1,-1,2,3T,-1,7,4,4,24T,...,11,7.0,10,5T,8.0,4,8T,-1,-1.0,-1
8,-1,7T,-1,11T,-1,13T,5,5,-1,4,...,4.0,5.0,-1,-1,15T,-1,-1,3,9,9T
9,-1,5T,-1,8T,-1,9,10T,-1,4,-1,...,-1.0,13T,-1.0,-1.0,-1,3.0,8T,-1,-1.0,-1.0


In [24]:
top = top_40

rank_data_40 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressor): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.4746549089068823
R-squared (RandomForest Regressor): 0.6024147984570738


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,25.325
13740,Hakeem Olajuwon,-1,C,0.0,24.850
13894,Dikembe Mutombo,1,C,39.0,24.760
13836,Michael Jordan,4,SG,6.0,9.115
13506,Tom Chambers,-1.0,PF,0.0,7.710
13903,Karl Malone,-1.0,PF,0.0,7.315
13866,Sean Higgins,-1.0,SF,0.0,7.125
13889,Tim Duncan,5T,PF,4.0,4.995
13605,David Robinson,3,C,10.0,4.485
13783,Mario Elie,-1,SF,0.0,2.135


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 11.91297979957806
R-squared (RandomForest Regressor): 0.2986380418989

,Player_name,Rank,Pos,Points_won,Predicted_points
13424,Gary Payton,3,PG,6.0,29.735
13182,Dikembe Mutombo,2,C,10.0,29.550
13269,Alonzo Mourning,1,C,89.0,29.160
13039,Hakeem Olajuwon,7T,C,1.0,28.110
13434,David Robinson,4,C,3.0,6.975
13488,Tim Duncan,5T,PF,2.0,6.845
13445,Kevin Garnett,7T,PF,1.0,6.515
13397,Scottie Pippen,7T,SF,1.0,5.350
13121,Bo Outlaw,-1,PF,0.0,5.180
13313,Brevin Knight,-1,PG,0.0,5.065


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 7.495049358974358
R-squared (RandomForest Regressor): 0.2269936581858

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,102.290
1046,Sekou Doumbouya,-1.0,PF,0.0,92.595
610,Michael Porter Jr.,-1.0,SF,0.0,80.775
1184,Nikola Jokić,-1.0,C,0.0,55.200
1026,Myles Turner,-1.0,C,0.0,38.500
992,Paul George,-1.0,PF,0.0,37.755
1212,Jayson Tatum,-1.0,SF,0.0,31.595
933,Al Horford,9,C,3.0,30.725
1275,Stephen Curry,-1.0,PG,0.0,26.865
1123,Draymond Green,10,PF,2.0,24.740


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 298.9463083743842
R-squared (RandomForest Regressor): 0.2974518159748

,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,69.230
76,Mac McClung,-1.0,SG,0.0,64.240
323,Brook Lopez,2.0,C,309.0,60.850
206,Nic Claxton,9T,C,7.0,54.545
392,Jacob Gilyard,-1.0,PG,0.0,37.430
184,Shaquille Harrison,-1.0,SG,0.0,32.155
137,Evan Mobley,3.0,PF,101.0,28.305
202,Marcus Smart,-1,PG,0.0,24.050
524,Bam Adebayo,5.0,C,18.0,17.205
497,Joel Embiid,9T,C,7.0,16.820


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,3,3T,-1,1,1,2,-1,1,6,...,1.0,2,3,2.0,15T,1.0,1.0,1.0,3.0,1.0
1,-1,2,5T,-1,-1,-1,-1,-1,13T,3,...,11,4,1,1,1.0,2.0,3.0,9.0,-1.0,-1.0
2,1,1,-1,1,-1,11,1,1,7,13T,...,7.0,1,2,5T,3,5.0,-1.0,-1.0,-1.0,2.0
3,4,7T,1,1,3T,11,-1,-1,6,-1.0,...,-1.0,-1.0,5.0,-1.0,6,-1,11,-1.0,-1.0,9T
4,-1.0,4,2,-1,-1,4,-1,-1,-1,-1,...,-1.0,5.0,-1.0,-1,-1.0,-1.0,-1.0,5.0,-1.0,-1.0
5,-1.0,5T,-1,3,-1,3,-1,9,-1,1,...,10.0,-1.0,4.0,3,8.0,-1,2,-1,-1.0,-1.0
6,-1.0,7T,-1,5T,2,-1,6,4,-1.0,-1,...,-1.0,15T,-1,-1.0,-1.0,-1.0,10.0,-1,-1.0,3.0
7,5T,7T,-1,2,3T,8,7,18T,4,4,...,13T,8,11T,-1.0,15T,6T,8T,6.0,9,-1
8,3,-1,7,11T,-1,13T,10T,5,4,16T,...,-1.0,-1.0,-1,-1,2,4,8T,-1,-1.0,5.0
9,-1,-1,-1,-1,-1,9,10T,-1,-1,24T,...,-1.0,-1.0,10,-1.0,-1,3.0,8T,-1,10,9T


In [25]:
top = top_50

rank_data_50 = pd.DataFrame()

for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressor): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.6103

,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,25.520
13894,Dikembe Mutombo,1,C,39.0,25.470
13740,Hakeem Olajuwon,-1,C,0.0,25.015
13506,Tom Chambers,-1.0,PF,0.0,10.805
13866,Sean Higgins,-1.0,SF,0.0,9.540
13836,Michael Jordan,4,SG,6.0,6.470
13903,Karl Malone,-1.0,PF,0.0,5.260
13889,Tim Duncan,5T,PF,4.0,3.990
13605,David Robinson,3,C,10.0,3.350
13783,Mario Elie,-1,SF,0.0,2.145


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

,Player_name,Rank,Pos,Points_won,Predicted_points
13424,Gary Payton,3,PG,6.0,29.635
13182,Dikembe Mutombo,2,C,10.0,29.460
13039,Hakeem Olajuwon,7T,C,1.0,28.970
13269,Alonzo Mourning,1,C,89.0,28.365
13434,David Robinson,4,C,3.0,8.475
13450,Theo Ratliff,5T,PF,2.0,7.420
13445,Kevin Garnett,7T,PF,1.0,6.915
13488,Tim Duncan,5T,PF,2.0,6.870
13237,Eddie Jones,7T,SF,1.0,6.370
13165,Patrick Ewing,-1,C,0.0,6.105


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,102.400
1046,Sekou Doumbouya,-1.0,PF,0.0,100.280
610,Michael Porter Jr.,-1.0,SF,0.0,91.395
1184,Nikola Jokić,-1.0,C,0.0,83.780
992,Paul George,-1.0,PF,0.0,36.315
1026,Myles Turner,-1.0,C,0.0,36.115
1275,Stephen Curry,-1.0,PG,0.0,31.930
1212,Jayson Tatum,-1.0,SF,0.0,30.875
933,Al Horford,9,C,3.0,25.800
813,Ricky Rubio,-1.0,PG,0.0,25.300


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,68.775
76,Mac McClung,-1.0,SG,0.0,57.210
323,Brook Lopez,2.0,C,309.0,54.060
206,Nic Claxton,9T,C,7.0,50.640
202,Marcus Smart,-1,PG,0.0,36.340
184,Shaquille Harrison,-1.0,SG,0.0,32.010
392,Jacob Gilyard,-1.0,PG,0.0,28.475
208,Walker Kessler,-1.0,C,0.0,23.640
75,Myles Turner,-1.0,C,0.0,19.265
137,Evan Mobley,3.0,PF,101.0,18.975


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,3,3T,-1,1,1,2,-1,1,-1,...,1.0,2,3,2.0,15T,1.0,1.0,1.0,3.0,1.0
1,1,2,-1,1,-1,-1,-1,1,13T,6,...,-1.0,4,1,1,1.0,2.0,3.0,9.0,-1.0,-1.0
2,-1,7T,5T,-1,-1,11,1,-1,7,3,...,11,1,2,5T,6,-1,-1.0,-1.0,-1.0,2.0
3,-1.0,1,1,1,-1,11,-1,-1,6,13T,...,7.0,-1.0,5.0,-1.0,3,5.0,11,-1.0,-1.0,9T
4,-1.0,4,2,-1,3T,4,-1,-1,-1,-1.0,...,-1.0,-1.0,-1.0,3,-1,-1.0,-1.0,5.0,-1.0,-1
5,4,5T,-1,3,-1,3,-1,9,-1,1,...,13T,5.0,-1,-1,15T,-1,2,-1,-1.0,-1.0
6,-1.0,7T,-1,5T,3T,8,6,18T,-1.0,-1,...,-1.0,8,11T,-1.0,2,4,10.0,-1,-1.0,-1.0
7,5T,5T,-1,11T,2,-1,7,4,8,4,...,-1.0,-1.0,4.0,-1,8.0,6T,8T,-1,-1.0,-1.0
8,3,7T,7,2,-1,13T,4,5,4,16T,...,10.0,15T,10,-1.0,-1.0,-1.0,8T,3,9,-1.0
9,-1,-1,8T,8T,-1,9,10T,-1,4,24T,...,-1.0,7.0,-1,-1.0,-1.0,-1,8T,-1,-1.0,3.0


In [26]:
rank_data_10.to_csv('RDF+RDF_Period_rank_data_10.csv', index=False)
rank_data_20.to_csv('RDF+RDF_Period_rank_data_20.csv', index=False)
rank_data_30.to_csv('RDF+RDF_Period_rank_data_30.csv', index=False)
rank_data_40.to_csv('RDF+RDF_Period_rank_data_40.csv', index=False)
rank_data_50.to_csv('RDF+RDF_Period_rank_data_50.csv', index=False)